In [2]:
disp('hello from MATLAB')

hello from MATLAB


In [4]:
disp('I am currently runing from isabelle s local computer')

I am currently runing from isabelle s local computer


# Exploring Jupyter Notebooks

# Outline of Tutorial

## 1) Congratulations on Opening This Notebook!

Beginner approach

    a) What this notebook will allow you to do: Make a copy and take you own notes!
    b) What NOT to do with this notebook: Running commands OUT OF ORDER
    c) How to use this notebook
    d) Text Editor to coopy your function and run them in Matlab or VS Code
    
## 2) This GitHub Repository 
    a) Requirements
    b) Set-up
    c) ReadME.md
    d) License
    e) Pdf to copy past and run directly in MATLAB locally 

## 3) Goal Workflow Description of a Hidden Markov Model Classifiers
    a) What is the theory behind this
    b) Sequential problems
    

## 4) GitKraken and how to use it in relation with this Notebook
    a) Control Version
    b) Build the different script versions 
    c) Be collaborative! :) 


## 5) Get the Data
    a) BIDS-EEG Format
    b) Use sample data from Coffey et al., 2017 on OSF.oi (check Open Science DOI)
    c) Use your own data set 
    
## 6) Classifier Folder
    a) Setup.m
    b) ExploreInput.m 
    c) CreateInputDatabase.m
    d) main.m
    e) main.m Functions
    f) Results directory
    g) Logprob directory
    h) figures directory
        

In [1]:
% Basics

    close all;
    clear all; 

    p = gcp('nocreate');
    if isempty(p)
    parpool(2);                                                               
    end
    
    delete('results/*.mat')

    rng('shuffle');                                                            
    params = setup; 
    
    totsizes = [50 100 200 500 1000 1500 2000 2300];
    avesizes = round(totsizes/4);
    
% Main

    for k=1:length(totsizes)
        
        tic
        
        params.totsize = totsizes(k);
        params.avesize = avesizes(k);
      
        subList = dir([params.ddir,'*.mat']);                                

        for i=1:length(subList)  

            % load subject ffrs

            sub = subList(i).name;
            load([params.ddir,sub]); 
            clsList = intersect(cls,cls);

            % train/test kfolds

            input = struct('ffrs',ffrs,'cls',cls);
            parfor j=1:length(clsList) 
            clsid = clsList(j);
            [trainfolds{j},testfolds{j}] = create_kfolds(input,clsid,params);
            end

            % move-average train folds

            parfor j=1:length(clsList)        
            trainfolds{j} = moving_average(trainfolds{j},params);           
            end

            % f0s

             for j=1:length(clsList)                                              
             trainfolds{j} = ffr2f0(trainfolds{j},params);                               
             testfolds{j} = ffr2f0(testfolds{j},params);                               
             end

            % rearrange kfolds

            [trainfolds,testfolds] = rearrange_kfolds(trainfolds,testfolds);

            % cross-validation

            pcls = [];
            rcls = [];
            plogps = [];

            for j=1:length(testfolds)

                trainfold = trainfolds{j};
                testfold = testfolds{j};
                [right pred logp] = hmm_modeling(trainfold,testfold,params);
                rcls = [rcls;right];
                pcls = [pcls;pred];
                plogps = [plogps;logp];

            end
            
            cm = confusionmat(rcls,pcls);
            sub
            cm = 100*cm/mean(sum(cm,1))

            % save results, clear vars, toc

            outname = [params.rdir,'sub',sub(1:end-4),'_nst',num2str(params.nst),...
            '_size',num2str(params.totsize),...
            '_ave',num2str(params.avesize)];   
            save(outname,'pcls','rcls','plogps');        
            clear trainfolds testfolds

        end
        
        toc
    
    end



Starting parallel pool (parpool) using the 'local' profile ...
> In verifyTimeZone (line 23)
  In datetime (line 515)
  In parallel.internal.cluster.DefaultProperties.getNewJobPropStruct (line 16)
  In parallel.internal.cluster.CJSSupport/buildJob (line 158)
  In parallel.cluster.CJSCluster/buildCommunicatingJob (line 465)
  In parallel.Cluster/createCommunicatingJob (line 92)
  In parallel.internal.pool.InteractiveClient>iCreateCommunicatingJob (line 715)
  In parallel.internal.pool.InteractiveClient/start (line 325)
  In parallel.Pool>iStartClient (line 593)
  In parallel.Pool.hBuildPool (line 472)
  In parallel.internal.pool.doParpool (line 18)
  In parpool (line 89)
connected to 2 workers.


Error using eval
Undefined function or variable 'setup'.



## Classifier Decoding Acuracy

## Figures


Now it is your turn to work try to run things